<a href="https://colab.research.google.com/github/datasci-rahul/personal/blob/main/Zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun May 12 16:04:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0              33W /  70W |   2373MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **ModelScope**

In [ ]:
!pip install modelscope==1.4.2
!pip install open_clip_torch
!pip install pytorch-lightning

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys

p = pipeline('text-to-video-synthesis', 'damo/text-to-video-synthesis')

In [ ]:
test_text = {
        'text': 'Darth Vader is surfing on waves',
    }
output_video_path = p(test_text,)[OutputKeys.OUTPUT_VIDEO]
print('output_video_path:', output_video_path)

In [ ]:
# Download result
from google.colab import files
files.download(output_video_path)

# **ZeroScope**

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install transformers accelerate torch


  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-s0rvkev_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-s0rvkev_
  Resolved https://github.com/huggingface/diffusers.git to commit 98ba18ba55f1f74b3d3b51fbe1db2c6978c9f9ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **zeroscope_v2 576w**

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

prompt = "Darth Vader is surfing on waves"
video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=24).frames
video_path = export_to_video(video_frames)

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 3)

# **zeroscope_v2 XL**

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

pipe.enable_vae_slicing()
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1) # disable if enough memory as this slows down significantly

prompt = "Darth Vader is surfing in waves"
video_frames = pipe(prompt, num_inference_steps=40, height=320, width=576, num_frames=36).frames
video_path = export_to_video(video_frames)

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
#!pip install opencv-python

In [ ]:
print(type(video_frames))

<class 'torch.Tensor'>


In [ ]:
video_frames = torch.from_numpy(video_frames)

In [ ]:
video_frames = video_frames.to(torch.uint8)

In [ ]:
import cv2
import torch
import numpy as np

def export_to_video(video_frames, output_video_path, fps):
    # Convert the tensor to a NumPy array
    video_frames = video_frames.numpy()

    # Convert the NumPy array to the appropriate data type
    video_frames = video_frames.astype(np.uint8)

    # Extract the height and width from the first frame
    h, w, c = video_frames[0][0].shape

    # Create a video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps=fps, frameSize=(w, h))

    # Write the frames to the video
    for i in range(video_frames.shape[1]):
        frame = video_frames[0][i]
        video_writer.write(frame)

    # Close the video writer
    video_writer.release()
video_path = export_to_video(video_frames, output_video_path="output.mp4", fps=24)